In [8]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.layers import Input, Dense

In [2]:
base_dir = 'dogs_breed'
valid_dir = 'valid'
train_dir = 'train'
train_path = os.path.join(base_dir, train_dir)
valid_path = os.path.join(base_dir, valid_dir)
print(train_path)

dogs_breed\train


In [3]:
model_VGG16 = VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)
model_VGG16.summary()

553467904/553467096 [==============================] - 181s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________

In [6]:
personal_model = Sequential()
for layer in model_VGG16.layers[:-1]:
    personal_model.add(layer)
    
personal_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [10]:
personal_model.add(Dense(units=len(os.listdir(train_path)), activation='softmax'))
personal_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [11]:
personal_model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
train_datagen_personal_vgg = ImageDataGenerator(rescale=1./255)
  
val_datagen_personal_vgg = ImageDataGenerator(rescale=1./255)

train_img_nb = 16469
valid_img_nb = 4111
batch_size = 20
epochs = 15

train_generator = train_datagen_personal_vgg.flow_from_directory(
    train_dir,  
    target_size=(224, 224),  
    batch_size=batch_size,
    classes=os.listdir(train_path)
)

valid_generator = val_datagen_personal_vgg.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    classes=os.listdir(train_path)
)

history = personal_model.fit(
    train_generator,
    steps_per_epoch=int(train_img_nb / batch_size),
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=int(valid_img_nb / batch_size),
)